In [ ]:
import numpy as np

f = open("instances/aaa1", "r")
content = f.read()
f.close()
#print(content)

lines = content.split("\n")

array = []

for line in lines:
    if line.startswith('#'): # filter lines of comment
        pass
    else :
        numbers = line.split(" ") 
        while numbers.count('') >0:
            numbers.remove('')
        for j in range(len(numbers)):
            numbers[j] = int(numbers[j])

        if numbers != []:
            array.append(numbers)

print(array)

nb_jobs = array[0][0] #number of jobs
nb_machines = array[0][1] #number of machines

print(nb_jobs)
print(nb_machines)

In [ ]:
machines = np.matrix(array[1:])[:,::2]
print(machines)

In [ ]:
durations = np.matrix(array[1:])[:,1::2]
print(durations)

In [ ]:
naive_makespan = np.sum(durations)

In [ ]:
list_r = []
for i in range(nb_machines):
    list_r.append("r"+str(i+1))

list_r

In [ ]:
from numpy.random import default_rng

list_solution = []

for i in range(nb_machines):
    rng = default_rng()
    list_ressource = rng.choice((1,nb_jobs), size=nb_jobs, replace=False).tolist()
    list_solution.append(list_ressource)
    
list_solution

In [ ]:
list_tuples = []
l = []

for i in range(len(list_solution)):
    for j in range(nb_jobs):
        job_number = list_solution[i][j]
        operation = machines[job_number-1,i]+1
#         print(job_number, operation)
        t = (job_number, operation)
        list_tuples.append(t)
        
list_tuples

In [ ]:

l = []

for i in range(len(list_solution)):
    list_tuples = []
    for j in range(nb_jobs):
        job_number = list_solution[i][j]
        operation = machines[job_number-1,i]+1
#         print(job_number, operation)
        t = (job_number, operation)
        list_tuples.append(t)
    l.append(list_tuples)

l

In [ ]:
machines[0,0]

In [ ]:
len(list_solution)

In [ ]:
import pandas as pd
df = pd.DataFrame(l, index=list_r)
df

In [ ]:
df_sol = pd.DataFrame([[(1, 1), (2, 2)], [(2, 1), (1, 2)], [(1, 3), (2, 3)]], index=list_r)
df_sol

In [ ]:
df_opt = pd.DataFrame([[(1, 1), (2, 2)], [(2, 1), (1, 2)], [(2, 3), (1, 3)]], index=list_r)
df_opt

## Séance 2

In [ ]:
stock = np.zeros(nb_machines)
stock

In [ ]:
repr_job = np.zeros((nb_jobs, nb_machines))
repr_job

In [ ]:
i,c = df.isin([(1,1)])
print (df.index[i], df.columns[c])

In [ ]:
t = (2,1)
c = df.columns[df.isin([t]).any()].tolist()[0]
i = df.index[df[c].isin([t])].tolist()[0]
c,i

In [ ]:
def index_col(tupl, df):
    c = df.columns[df.isin([t]).any()].tolist()[0]
    str_i = df.index[df[c].isin([t])].tolist()[0]
    index = str_i.replace("r", "")
    index = int(index)
    return str_i, index-1, c

In [ ]:
print(df_sol)
print(durations)

In [ ]:
# stock = np.zeros(nb_machines)
repr_job = np.zeros((nb_jobs, nb_machines))
counter_job = np.zeros(nb_jobs)
counter_ress = np.zeros(nb_machines)
ready_job = np.zeros(nb_jobs)
ready_ress = np.zeros(nb_machines)

while any(x != nb_jobs for x in counter_ress):
    for job in range(1, nb_jobs+1):
        for op in range(1, nb_machines+1):
#             print("============================")
            t = (job,op)
#             print(t)
            ind, ressource, col =  index_col(t, df_opt)
#             print("DF : index : " + ind + " colonne " +str(col))
#             print(counter_job[job-1])
#             print(counter_ress[ressource])
            if((col == counter_ress[ressource])&(op-1 == counter_job[job-1])):
    
#                 print("job prêt : " + str(ready_job))
#                 print("ressource prête : " + str(ready_ress))
#                 print("DO job "+str(job)+ "/ op "+ str(op))
                
#                 if counter_job[job-1] == 0:         Verify if operation == 0
#                     repr_job[job-1][op-1] = 0
#                 else:
#                 print("ressource duration : " + str(ready_ress[ressource]))
#                 print("job duration : " + str(ready_job[job-1]))
                repr_job[job-1][op-1] =  max(ready_job[job-1], ready_ress[ressource])
                
                
                ready_ress[ressource] = ready_ress[ressource] + durations[job-1, op-1] + ready_job[job-1]
#                 ready_job[job-1] = ready_job[job-1] + durations[job-1, op-1]
                ready_job[job-1] = repr_job[job-1][op-1] + + durations[job-1, op-1]
#                 print(repr_job)
                counter_ress[ressource]+=1
                counter_job[job-1]+=1
#                 print('état ressource :[%s]' % ', '.join(map(str, counter_ress)))
#                 print('état job :[%s]' % ', '.join(map(str, counter_job)))
#             else:
#                 print("STOP job "+str(job)+ "/ op "+ str(op))
                

print(repr_job)

In [ ]:
list_jobs=[]
[list_jobs.append('J'+str(x)) for x in range(1, nb_jobs+1)]
list_jobs

In [ ]:
df_repr_detail = pd.DataFrame(data=repr_job, index=list_jobs, columns=range(1,4))
df_repr_detail

In [ ]:
print(machines)
print(durations)

In [ ]:
list_gantt = []
for i in range(nb_jobs):
    for j in range(nb_machines):
        list_gantt.append(['r' + str(machines[i,j]+1), 
                          str(today + timedelta(days=repr_job[i][j])),
                              str(today +  timedelta(days=repr_job[i][j])+ timedelta(days=int(durations[i,j]))), 
                                  'Job'+str(i+1)])
list_gantt

In [ ]:
from plotly.figure_factory import create_gantt
import pandas as pd
from datetime import date
from datetime import timedelta
import datetime

today = date.today()

df = pd.DataFrame(list_gantt,
                  columns=['Task', 'Start', 'Finish', 'Resource'])

fig = create_gantt(df, index_col='Resource', show_colorbar=True,
                      group_tasks=True)

fig.show()

In [ ]:
 
today + timedelta(days=1) 

In [ ]:
 str(date.today() + timedelta(days=1))

In [ ]:
all(x == 2 for x in foobars)

In [ ]:
foobars = [2,2,2]

In [ ]:
repr_job[i][0]

In [ ]:
durations[1, 0]

In [ ]:
durations

In [ ]:
df.isin([(1,1)])
df.index[df[c].isin([(1,3)])].tolist()

In [ ]:
i, c = np.where(df == (1,1))

print ((df.index[i].values, df.columns[c].values))

In [ ]:
a = df.iloc[0].values
for t in a:
    if(t[1] == 1):
        b = t
b

In [ ]:
print(a)

In [ ]:
df[0].values